In [ ]:
#!/usr/bin/env python

from fanuc_pcdk_client import FanucPCDKClient
from robot_loc_world import RobotLocWorld
from robot_loc_joint import RobotLocJoint
from time import sleep
from copy import deepcopy
import sys

ROBOT_SPEED = 30
ROBOT_SPEED_LINEAR = 45

In [ ]:
# Robot init
robot = FanucPCDKClient()
robot.connect()

# Run teach pendant program.
robot.run_program('PCDK')
robot.reset_alarms()

# Adjust the speed of the robot
robot.set_speed(ROBOT_SPEED)

In [10]:
valid_letters = ['s', 'l', 'o', 'w', 'u', 'b']

In [22]:
# Trajectories for each letter

# Pen location

# Pick up pen traj

# Return pen traj - Not actually needed

# S traj


# L traj


# O traj


# W traj


# U traj


# B traj

# trajectories
trajectories = {'s': [], 'l': [], 'o': [], 'w': [], 'u': [], 'b': []}

In [23]:
def adjust_traj(letter):
    # Use function to adjust trajectory to shift
    # Maybe use anchor point for writing each letter (top left or right or something)
    pass

In [24]:
def pickup_pen():
    # Use to complete pen pick trajectory
    pass

In [25]:
def execute_writing(word):
    print("\n\n===============    STARTING WRITING    ===============")
    
    pickup_pen()
    
    for letter in word:
        
        traj = trajectories[letter]
        
        for pose in traj:
            robot.move(pose, move_linear=True, speed=ROBOT_SPEED_LINEAR)
    

In [26]:
terminate = False
user_input = 0
started = False
while not terminate:
    try:
        if(not started):
            print("===============  WELCOME TO THE ROBOTIC (LIMTIED) TYPEWRITER  ===============\n")
            started = True
            
        user_input = (raw_input("What 3-letter word would you like to write? (press q to quit)\
                        \nValid letters include S L O W U B\
                        \n> "))
        
        if(not (user_input == 'q')):
            if (len(user_input) > 3):
                raise ValueError
                
            for letter in user_input:
                if(letter.lower() not in valid_letters):
                    raise ValueError
                
        if(user_input == 'q'):
            print("\n===============    TERMINATING WRITING    ===============")
            terminate = True
            break
        else:
            '''
            ACTUAL WRITING STUFF GOES HERE
            '''
            execute_writing(user_input)
            
    except ValueError:
        print("Invalid word, please try again")

===============  WELCOME TO THE ROBOTIC (LIMTIED) TYPEWRITER  ===============

What 3-letter word would you like to write? (press q to quit)                        
Valid letters include S L O W U B                        
> q

===============    TERMINATING WRITING    ===============
